# 模型的存储与加载

有2种方法
1. 生成chekpoint file
2. 生成图协议文件

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import os

## 1.ckpt加载

### 1.1 定义模型

In [2]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

In [3]:
mnist_data_folder = '/home/jimo/workspace/temp/mnist-data/' # 可以先下好，免得下载慢
mnist = input_data.read_data_sets(mnist_data_folder, one_hot=True)
trX,trY,teX,teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

X = tf.placeholder('float',[None,784])
Y = tf.placeholder('float',[None,10])
# 初始化权重参数
w_h = init_weights([784,625])
w_h2 = init_weights([625, 625])
w_o = init_weights([625, 10])

Extracting /home/jimo/workspace/temp/mnist-data/train-images-idx3-ubyte.gz
Extracting /home/jimo/workspace/temp/mnist-data/train-labels-idx1-ubyte.gz
Extracting /home/jimo/workspace/temp/mnist-data/t10k-images-idx3-ubyte.gz
Extracting /home/jimo/workspace/temp/mnist-data/t10k-labels-idx1-ubyte.gz


In [4]:
# 定义模型
def model(X,w_h,w_h2,w_o,p_keep_input,p_keep_hidden):
    # 第一个全连接层
    X = tf.nn.dropout(X,p_keep_input)
    h = tf.nn.relu(tf.matmul(X,w_h))
    h = tf.nn.dropout(h,p_keep_hidden)
    # 第二个全连接层
    h2 = tf.nn.relu(tf.matmul(h,w_h2))
    h2 = tf.nn.dropout(h2,p_keep_hidden)
    
    return tf.matmul(h2,w_o)

In [5]:
# 生成模型并预测
p_keep_input = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

### 1.2 训练及存储模型

In [9]:
ckpt_dir = './03_ckpt_dir'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

global_step = tf.Variable(0,name='global_step',trainable=False)
# 在saver前声明的变量才会存储
saver = tf.train.Saver()
non_storage_variable = tf.Variable(777)

# 训练
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    start = global_step.eval() # get last global_step
    print("Start from:", start)

    for i in range(start, 100):
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)):
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],
                                          p_keep_input: 0.8, p_keep_hidden: 0.5})

        global_step.assign(i).eval() # set and update(eval) global_step with index, i
        saver.save(sess, ckpt_dir + "/model.ckpt", global_step=global_step)
        print(i, np.mean(np.argmax(teY, axis=1) ==
                         sess.run(predict_op, feed_dict={X: teX, Y: teY,
                                                         p_keep_input: 1.0,
                                                         p_keep_hidden: 1.0})))

./03_ckpt_dir/model.ckpt-3
INFO:tensorflow:Restoring parameters from ./03_ckpt_dir/model.ckpt-3


NotFoundError: Key global_step_3 not found in checkpoint
	 [[Node: save_3/RestoreV2_15 = RestoreV2[dtypes=[DT_INT32], _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_save_3/Const_0_0, save_3/RestoreV2_15/tensor_names, save_3/RestoreV2_15/shape_and_slices)]]

Caused by op 'save_3/RestoreV2_15', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/usr/lib/python

### 1.3 恢复模型

In [ ]:
    ckpt = tf.train.get_checkpoint_state(ckpt_dir)
    if ckpt and ckpt.model_checkpoint_path:
        print(ckpt.model_checkpoint_path)
        saver.restore(sess, ckpt.model_checkpoint_path) # restore all variables

## 2.图模型

In [10]:
v = tf.Variable(0,name='my_variable')
sess = tf.Session()
tf.train.write_graph(sess.graph_def,'/tmp/tfmodel','train.pbtxt')

'/tmp/tfmodel/train.pbtxt'

In [12]:
with tf.Session() as _sess:
    with tf.gfile.FastGFile('/tmp/tfmodel/train.pbtxt','rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _sess.graph.as_default()
        tf.import_graph_def(graph_def,name='tfgraph')

DecodeError: Error parsing message